In [1]:
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset
import spacy
import argparse
from utils_transform import *
from transformer import *
import pdb
import time, datetime
import sys
from collections import namedtuple
import torch
from eval_lib import *

In [4]:
ArgTuple = namedtuple('args', 'size')
args = ArgTuple(size="mid")
args.size
params = Params(args)
params.vocab_size

10000

In [5]:
print("Train: %s, Val: %s, test: %s" % (params.train_csv, params.val_csv, params.test_csv))
print("Vocab size: %d" % (params.vocab_size))
train, val, test, TR, EN = load_train_val_test_datasets(params)
pad_idx = EN.vocab.stoi[PAD]

Train: train_200k.csv, Val: val_10k.csv, test: test_10k.csv
Vocab size: 10000
Building vocab...
TR vocab size: 9718, EN vocab size: 10004
Done building vocab


In [6]:
rev_tokenize_en = lambda tokenized: [EN.vocab.itos[i] for i in tokenized]
rev_tokenize_tr = lambda tokenized: [TR.vocab.itos[i] for i in tokenized]


In [7]:
batch_src = torch.tensor([[  5,   8,  19,  56, 539,   3,  46, 328,  39,   8,   4],
        [  6,  43,   7, 506, 353,  45, 270,  15,   9,  24,   2],
        [195, 137,  67, 249,  34,   3,  70,  55, 152,  16,   2],
        [ 94,  21,  33,  14,  86, 132, 293,  66, 190, 227,   2],
        [241, 308,  74, 359,  80,  48,  27,  64,  17,  14,   4],
        [115,   7, 142,  40, 239,  74, 271,  58, 117,  22,   2],
        [ 93,  62, 329,  33,  89,   5,   2,  96,  39, 164,   2],
        [242,  42,  54, 124,   3,  63,  10, 122,  30, 291,   2],
        [ 60,  11, 236, 124, 130, 158, 338, 416,  20,   8,   2],
        [685, 370,  94, 305, 116,  20,   8,  17, 160,  19,   4],
        [ 18, 287,  92,   9, 220, 179, 123, 297, 341,  86,   2],
        [ 25, 160,  97,  26, 149,   7, 227,  16,  81,  34,  13],
        [193,  12, 532,  31, 116,  21, 126,  16,  98,  11,   2],
        [145,  87,  46,  98, 224,   3, 101,  48,  17,  14,   4],
        [221,  62,  23,  27,  11,   3, 230,   7,  52, 365,   4],
        [553, 295, 141, 222, 274,  73, 105, 127, 185, 120,   2],
        [ 47, 262,  56,  87, 136, 618,  58,  35,  15,  34,   4],
        [  6,  57,  35,   6,  71,   3,  46, 298, 282,  32,   2],
        [ 60,  74,  66,  71,  70, 287, 144,  16,  98,  11,   2],
        [  5,  24,  40,  51,  74, 108, 163,  15,  19, 149,   2]])
batch_context = torch.tensor([[159,   7,  63, 170,  93,  70,  21, 108, 178,  84, 170, 441,  38, 221,
         105,  59,  37,  24,   2,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [189,  36,  12, 399,  20,  14, 106,  81,  80,  48, 167,  31,  15,  45,
          18,  43,  42,  84, 119, 106,  81,  81,  22,   4,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [ 65, 571,  18, 112,  14,   2,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [  5,  71, 158, 132, 544,   2,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [ 25,  21, 458,  75,  50,   2,   2,   2,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [ 63,  10, 122,  30, 291,   2,   2,   2,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [ 52,   5,   2,  29, 305,  69,  83,   8,  15,  19,   6,  95,  73,   4,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [  6,  19, 257,  18, 233, 418,  60,  11,  18,  55,  38,   2,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [  6,  80,  38, 356,  11,  88, 131, 242, 374,  16, 168, 220,  50, 112,
          22,   2,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [  6, 330, 131,  52, 196, 443, 150,  80, 304,  59,  37, 305, 116,  45,
          20,   8,  26,   2,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [ 44,   5, 215,  59,  12, 662,  14,  63, 448, 125,  74, 359, 136, 219,
           3, 162,  38,  17,  32,   4,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [114, 112, 447, 146, 152,  37,  79, 133,   6,  24,  14,  93,  18, 149,
          97,  15, 149, 202,  20,   8,   2,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [113,  57,   3, 380,  33,  21,  59, 486,  19, 296, 116,  42,  84,  81,
          90, 220, 201,  28,  27,  22,   2,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [ 56,  36, 208,  28,  24,   4,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [ 44,  88, 189, 263, 127, 298,  61,  58,  35,   2,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [ 60, 250,  87,  79, 153, 172,  37,  79, 133, 210,  16,  22,   2,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [311,  75,  91, 478, 429,   3, 158,  62, 534,  42, 111,  54, 130, 295,
         141, 142, 438, 340,  12,  14, 593,   2,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [ 56,  49, 265,   3,   5,  24,   9,  73, 323,  42, 455, 142,  40, 239,
         317,  23, 213, 140,  92,  49,  27,  11,   3,   5,  24,  14,  46, 431,
         178,  36,  27,  11,   2,   1,   1,   1,   1,   1,   1],
        [ 18, 287,  92,   6, 344,  12, 107, 136,  42,  34, 496, 151,  82,   5,
          95,  50, 284,   3, 228,  55,   6,  24, 138,  91, 185,  23,  12, 283,
          11,  24,  14,  88,   5, 186,  20,   8, 184,  29,   2],
        [  6,  82,  56,  87, 136, 116,  21,  59,  37,  24,   4,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1]])


In [8]:
print(len(TR.vocab), len(EN.vocab))
pad_idx = EN.vocab.stoi[PAD]

9718 10004


In [30]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, time, copy
import pdb

class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, batch):
        "Take in and process masked src and target sequences."
        src, tgt, src_mask, tgt_mask = batch.src, batch.trg, batch.src_mask, batch.trg_mask
        src_context, src_context_mask = batch.src_context, batch.src_context_mask
        encoder_output = self.encode(batch)
        return self.decode(encoder_output, src_mask,
                            tgt, tgt_mask)
    
    def encode(self, batch):
        if self.encoder.use_context:
            return self.encoder(
                self.src_embed(batch.src), batch.src_mask, 
                self.src_embed(batch.src_context), batch.src_context_mask)
        else:
            return self.encoder(self.src_embed(batch.src), batch.src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        embedded = self.tgt_embed(tgt)
        return self.decoder(embedded, memory, src_mask, tgt_mask)

class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = nn.LayerNorm(layer.size, eps=1e-6)
        self.use_context = False
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class CombinationLayer(nn.Module):
    '''Combines the outputs of the source encoder and the context encoder.
    '''
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(CombinationLayer, self).__init__()
        self.self_attn = self_attn
        self.context_attn = copy.deepcopy(self_attn)
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
        self.size = size
        self.w = nn.Linear(2 * self.size, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, src_mask, context, context_mask):
        "Follow Figure 1 (right) for connections."

        # From the source encoder, there's a self-connection sublayer.
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, src_mask))

        # Fomr the context encoder.
        context = self.sublayer[1](x, lambda x: self.context_attn(x, context, context, context_mask))

        # Append context and X together
        x_and_context = torch.cat([x, context], dim=2) # [batch x len x 2*size]
        g = self.sigmoid(self.w(x_and_context))

        print(g)

        # Gated sum
        gated_sum = g * x + (1 - g) * context
        return self.sublayer[2](gated_sum, self.feed_forward)


class EncoderWithContext(nn.Module):
    def __init__(self, layer, N):
        super(EncoderWithContext, self).__init__()
        # N "regular" layers. Share embeddings
        self.layers = clones(layer, N)
        self.norm = nn.LayerNorm(layer.size, eps=1e-6)
        self.combination_layer = CombinationLayer(
            layer.size, copy.deepcopy(layer.self_attn), 
            copy.deepcopy(layer.feed_forward), layer.dropout)
        self.use_context= True
        
    def forward(self, x, mask, src_context, context_mask):
        "Pass the input (and mask) through each layer in turn."

        # The src only goes through N - 1 layers.
        for layer in self.layers[:-1]:
            x = layer(x, mask)

        for layer in self.layers:
            src_context = layer(src_context, context_mask) 

        out = self.combination_layer(x, mask, src_context, context_mask)
        return self.norm(out)



class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = nn.LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size
        self.dropout = dropout

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = nn.LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn 
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction='sum')
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.long().data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 1: # Empty tensors are dim 1 in Pytorch 0.4+
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        assert(not(true_dist.requires_grad))
        return self.criterion(x, true_dist)

def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    model.src_embed[0].lut.weight = model.tgt_embed[0].lut.weight
    # model.generator.lut.weight = model.tgt_embed[0].lut.weight
    return model

def make_context_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        EncoderWithContext(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    model.src_embed[0].lut.weight = model.tgt_embed[0].lut.weight
    # model.generator.lut.weight = model.tgt_embed[0].lut.weight
    return model

In [21]:
def log_likelihood(model, batch, pad_idx):
    """Calculates the log likelihood of a batch, given the model.
    """
    memory = model.encode(batch)
    total_prob = torch.zeros(batch.trg_y.size(0))
    for i in range(0, batch.trg_y.size(1)): # trg_len
        y_prev = batch.trg[:, :i + 1]
        out = model.decode(memory, batch.src_mask, 
                           y_prev.clone().detach(),
                           (subsequent_mask(y_prev.size(1))
                                    .type_as(batch.src.data)))
        probs = model.generator(out[:, -1]) # batch x vocab
        trg_index = batch.trg_y[:, i]
        prob_of_trg = probs.gather(1, trg_index.view(-1,1)) # not sure about this
        pad_mask = (trg_index != pad_idx).to(dtype=torch.float32)
        total_prob += (prob_of_trg.squeeze()) * pad_mask
        if i == 0:
            init_probs = total_prob
    return total_prob, init_probs

def greedy_decode(model, batch, max_len, start_symbol):
    src = batch.src, src_mask = batch.src_mask # This is just wrong lol
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    total_prob = 0.0
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           ys.clone().detach(),
                           (subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        max_prob, next_word = torch.max(prob, dim = 1)
        total_prob += max_prob.data
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

def beam_decode(model, src, src_mask, src_context, pad_idx, max_len, start_symbol, end_symbol, k=5):
    """Beam decoder.
    """
    batch = Batch(src.unsqueeze(0), src_mask.unsqueeze(0), src_context.unsqueeze(0), pad_idx)
    memory = model.encode(batch)
    ys = torch.ones(1,1).fill_(start_symbol).type_as(src.data)
    hypotheses = [(ys, 0.0)]
    for i in range(max_len):
      candidates_at_length = []
      for hypothesis, previous_prob in hypotheses:
        if hypothesis[0, -1] == end_symbol:
          candidates_at_length.append((hypothesis, previous_prob))
        else:
          # feed through model
          out = model.decode(memory, src_mask, 
                               hypothesis.clone().detach(),
                               (subsequent_mask(hypothesis.size(1))
                                        .type_as(src.data)))
          probs = model.generator(out[:, -1])
          # Keep track of top k predictions for each candidates
          top_probs, predictions_at_step = torch.topk(probs, k, dim=1)
          new_hypotheses = [torch.cat([hypothesis.clone(), pred.reshape(1,1)], dim=1) for pred in predictions_at_step.flatten()]
          new_probs = top_probs.flatten().data + previous_prob
          candidates_at_length = candidates_at_length + list(zip(new_hypotheses, new_probs))
      hypotheses = sorted(candidates_at_length, key = lambda x: x[1], reverse=True)[:k]
    return hypotheses[0][0].squeeze() # change to 1D tensor

def eval_bleu(pad_idx, eval_iter, model, max_len, start_symbol, end_symbol, rev_tokenize_trg, bpemb_en):
  """Calculates average BLEU score of a model on some validation iterator.
  """
  bleus = []
  for old_batch in eval_iter:
    batch = rebatch(pad_idx, old_batch)
    for i in range(batch.src.size(0)): # batch_size
      hypothesis = beam_decode(model, batch.src[i], batch.src_mask[i], batch.src_context[i],
       pad_idx, max_len, start_symbol, end_symbol, k=5)[1:-1] # cut off SOS, EOS
      targets = batch.trg_y[i, :-1] # Doesn't have SOS. Cut off EOS
      hyp_str = bpemb_en.decode(rev_tokenize_trg(hypothesis))
      trg_str = bpemb_en.decode(rev_tokenize_trg(targets))
      bleu = get_moses_multi_bleu([hyp_str], [trg_str])

      if bleu == None: # Error
        print(i, hyp_str, trg_str)
      else:
        print(bleu, hyp_str, trg_str)
        bleus.append(bleu)
  return sum(bleus) / len(bleus)

def eval_accuracy_helper(pad_idx, eval_iter, model):
  """ Helper function that calculates how well the model chooses the correct
  pronoun on the data in an iterator.
  """
  n_correct = 0.0
  n_total = 0.0
  log_diff = 0.0
  for b in eval_iter:
    batch_correct, batch_incorrect = rebatch_for_eval(pad_idx, b)
    res1 = log_likelihood(model, batch_correct, pad_idx)
    res2 = log_likelihood(model, batch_incorrect, pad_idx)
    probs = torch.stack([
      res1[0], 
      res2[0]], dim=1) # n x 2
    log_diff += sum(probs[:,0] - probs[:,1]).item()
    correct = probs[:, 0] > probs[:, 1] # should assign higher probability to the left
    print("sum of differences in log probs: ", probs[:, 0] - probs[:, 1])
    n_correct += torch.sum(correct).item()
    n_total += correct.size(0)
  print("Correct: %d / %d = %f" % (n_correct, n_total, (n_correct / n_total)))
  print("Log difference %f" % log_diff)
  print("Log difference for first terms %f" %  sum(res1[1] - res2[1]).item())
  return

def eval_accuracy(pad_idx, path_to_test_set, model, TR, EN):
  """Calculates how well the model chooses the correct pronoun, given
  a dataset in TSV form. 
  """
  full_path = "data/%s" % (path_to_test_set)
  # Must be in order
  # !!! IMPT note there are five fields
  data_fields = [
  ('src_context', TR), ('src', TR),
  ('trg_context', EN), ('trg_correct', EN), ('trg_incorrect', EN)]
  print('Evaluating discriminative dataset: %s ' % (full_path))
  test = TabularDataset(
    full_path,
    format='tsv', 
    fields=data_fields)

  test_iter = Iterator(
    test, batch_size=100, sort_key=lambda x: 1, repeat=False, train=False)
  eval_accuracy_helper(pad_idx, test_iter, model)


In [11]:
def load(path, tr_voc, en_voc, use_context):
    """Loads a trained model from memory for evaluation.
    """
    if use_context:
      model = make_context_model(tr_voc, en_voc, N=6)
    else:
      model = make_model(tr_voc, en_voc, N=6)
    model.load_state_dict(torch.load(path, map_location='cpu'))
    model.eval()
    return model

In [12]:
context_model = load('models/0227_context_200k_debugged_40.pt', len(TR.vocab), len(EN.vocab), True)

In [13]:
eval_accuracy(pad_idx, "female_subject.tsv", context_model, TR, EN)

Evaluating discriminative dataset: data/female_subject.tsv 
tensor([[[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         

sum of differences in log probs:  tensor([-0.5260, -0.5018, -0.8249, -0.9568, -0.7576, -0.4453, -0.8437, -0.8402,
        -0.8317, -0.6583, -0.5270, -0.6199, -1.1213, -0.9185, -0.1819, -1.1137,
        -0.6629, -0.5200, -0.6935, -0.8984, -0.8051, -0.9063, -0.8183, -0.6056,
        -0.5316, -1.0984, -1.0953, -0.9057, -1.0835, -0.8377, -0.7518, -1.0776,
        -1.7340, -0.5819, -0.6551, -0.6246, -0.3880, -0.6439, -0.7788, -0.8302],
       grad_fn=<SubBackward0>)
Correct: 0 / 40 = 0.000000
Log difference -31.196846
Log difference for first terms -31.196846


In [18]:
model_baseline = load('models/0220_baseline_200k_100.pt', len(TR.vocab), len(EN.vocab), False)

In [19]:
eval_accuracy(pad_idx, "female_subject.tsv", model_baseline, TR, EN)

Evaluating discriminative dataset: data/female_subject.tsv 
sum of differences in log probs:  tensor([-0.8280, -1.6794, -0.8245, -1.8511, -1.0720, -0.7715, -0.5312, -1.0224,
        -1.1075, -1.3326, -1.1280, -1.0859, -1.4362, -1.0091, -1.0416, -1.1151,
        -0.7611, -0.1875, -1.2887, -1.1325, -0.8460, -1.2922, -2.0314, -0.9724,
        -1.1152, -1.5892, -2.1911, -1.3357, -1.3766, -1.6188, -1.1225, -1.0069,
        -2.4823, -1.0564, -1.1161, -1.0570, -1.0841, -0.8606, -1.1548, -1.4232],
       grad_fn=<SubBackward0>)
Correct: 0 / 40 = 0.000000
Log difference -47.938416
Log difference for first terms -47.938416


In [31]:
new_context = load('models/0301_context_boc_200k_10.pt', len(TR.vocab), len(EN.vocab), True)

In [32]:
eval_accuracy(pad_idx, "female_subject.tsv", new_context, TR, EN)

Evaluating discriminative dataset: data/female_subject.tsv 
tensor([[[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [1.],
         [1.],
         

sum of differences in log probs:  tensor([-1.1688, -1.3411, -0.4435, -1.4115, -0.4737,  0.4368, -1.7191,  0.9614,
        -1.5919, -2.4081, -0.7998, -1.1531, -0.2525, -0.3300, -1.4130, -0.8722,
        -1.3525, -1.6742, -2.3171, -0.7716, -1.1659, -3.0730, -0.3808, -1.0429,
        -0.6962,  0.7966, -1.2891, -0.2843, -0.4878, -0.2663, -0.0737,  0.7109,
        -0.4005, -0.1491,  0.0211, -2.0143, -0.0599, -1.3845, -0.7994, -0.7486],
       grad_fn=<SubBackward0>)
Correct: 5 / 40 = 0.125000
Log difference -32.883102
Log difference for first terms -32.883102


In [33]:
len(TR.vocab)

9718